# Notebook For Nevergrad Example Storage 

## Simple Optimization

In [1]:
from nevergrad.optimization import optimizerlib

In [2]:
def square(x):
    return sum((x - .5)**2)

In [3]:
optimizer = optimizerlib.OnePlusOne(instrumentation=2, budget=100)

In [4]:
# alternatively, you can use optimizerlib.registry which is a dict containing all optimizer classes
recommendation = optimizer.optimize(square)
print(recommendation)  # optimal args and kwargs

Candidate(args=(array([0.50094495, 0.49980454]),), kwargs={})


#### Full List of optimizers:

In [5]:
from nevergrad.optimization import registry
print(sorted(registry.keys()))

['ASCMA2PDEthird', 'ASCMADEQRthird', 'ASCMADEthird', 'AlmostRotationInvariantDE', 'AlmostRotationInvariantDEAndBigPop', 'BO', 'BPRotationInvariantDE', 'CM', 'CMA', 'CMandAS', 'CMandAS2', 'CauchyLHSSearch', 'CauchyOnePlusOne', 'CauchyRandomSearch', 'CauchyScrHammersleySearch', 'Cobyla', 'DE', 'DiagonalCMA', 'DiscreteOnePlusOne', 'DoubleFastGADiscreteOnePlusOne', 'DoubleFastGAOptimisticNoisyDiscreteOnePlusOne', 'EDA', 'FastGADiscreteOnePlusOne', 'FastGANoisyDiscreteOnePlusOne', 'FastGAOptimisticNoisyDiscreteOnePlusOne', 'HaltonSearch', 'HaltonSearchPlusMiddlePoint', 'HammersleySearch', 'HammersleySearchPlusMiddlePoint', 'LBO', 'LHSSearch', 'LargeHaltonSearch', 'LargeHaltonSearchPlusMiddlePoint', 'LargeHammersleySearch', 'LargeHammersleySearchPlusMiddlePoint', 'LargeScrHaltonSearch', 'LargeScrHaltonSearchPlusMiddlePoint', 'LargeScrHammersleySearch', 'LargeScrHammersleySearchPlusMiddlePoint', 'LargerScaleRandomSearchPlusMiddlePoint', 'LhsDE', 'MEDA', 'MPCEDA', 'MicroCMA', 'MidQRBO', 'Milli

## Instrumentation

#### instrumentation=n is a shortcut to state that the function has only one variable, of dimension n, Defining the following instrumentation instead will optimize on both x and y.

recommendation holds the optimal attributes args and kwargs found by the optimizer for the provided function. In this example, the optimal value will be found in recommendation.args[0] and will be a np.ndarray of size 2.

instrumentation=n is a shortcut to state that the function has only one variable, of dimension n, Defining the following instrumentation instead will optimize on both x and y.

In [6]:
from nevergrad import instrumentation as inst

In [7]:
def square(x, y=12):
    return sum((x - .5)**2) + abs(y)

In [8]:
instrum = inst.Instrumentation(inst.var.Array(2), y=inst.var.Array(1).asscalar())
optimizer = optimizerlib.OnePlusOne(instrumentation=instrum, budget=100)
recommendation = optimizer.optimize(square)
print(recommendation)

Candidate(args=(array([0.48147511, 0.54972631]),), kwargs={'y': -1.1967642133421633e-05})


### Ask and Tell interface


- ask: suggest a candidate on which to evaluate the function to optimize.
- tell: for updated the optimizer with the value of the function for a candidate.
- provide_recommendation: returns the candidate the algorithms considers the best. For most optimization algorithms in the platform, they can be called in arbitrary order - asynchronous optimization is OK. Some algorithms (with class attribute no_parallelization=True however do not support this.



In [9]:
for _ in range(optimizer.budget):
    x = optimizer.ask()
    value = square(*x.args, **x.kwargs)
    optimizer.tell(x, value)


In [10]:
recommendation = optimizer.provide_recommendation()
recommendation

Candidate(args=(array([0.48164839, 0.54957443]),), kwargs={'y': -4.8796638032245744e-08}, data=[ 4.81648388e-01  5.49574426e-01 -4.87966380e-08])

## Executor

In [11]:
from concurrent import futures
optimizer = optimizerlib.OnePlusOne(instrumentation=instrum, budget=100, num_workers=5)
with futures.ProcessPoolExecutor(max_workers=optimizer.num_workers) as executor:
    recommendation = optimizer.optimize(square, executor=executor, batch_mode=False)

In [12]:
futures.ProcessPoolExecutor(max_workers=optimizer.num_workers)

In [13]:
from concurrent.futures import ThreadPoolExecutor
from time import sleep
 
def return_after_5_secs(message):
    sleep(5)
    return message
 
pool = ThreadPoolExecutor(3)
 
future = pool.submit(return_after_5_secs, ("hello"))
print(future.done())
sleep(5)
print(future.done())
print(future.result())

False
True
hello
